## Globale Temperaturänderungen und Gefahren

### Teil 2
- Daten zu Gefahren einlesen, analysieren und aufbereiten

---

In [247]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

pd.set_option('display.max_colwidth', None)  # Spaltenbreitenberschränkung aufheben
# pd.set_option('display.max_rows', 60)  # Maximale Zeilenausgabe setzen


---
# Gefahrendaten einlesen und aufbereiten
## 1. Einlesen
Ausgewählte Spalten werden aus 2023_Cities_Climate_Hazards_20250506.csv ins das Dataframe `hazard_df` geladen:
| Spalte | Beschreibung | Anmerkung | Umbenennung |
|-|-|-|-|
| CDP Region | betroffene Region |  | region |
| Country/Area | betroffenes Land / Gebiet |  | country
| City | betroffene Stadt | unvollständig | city |
| Organization Name | evaluierende Organization |  | *fällt weg / Lückenfüller für city* |
| Climate-related hazards | Auswirkung der Temperaturerhöhung |  | hazards |
| Current magnitude of impact of hazard | Ausmaß der Auswirkung | von Low bis High | hazard_impact |
| Current probability of hazard | Gefahrenwahrscheinlichkeit | von Low bis High | hazard_probability |
| Proportion of population exposed to hazard | betroffener Bevölkerungsanteil | in % | hazard_exposion |
| Expected future change in hazard intensity | prognostizierte Veränderung der Intensität | steigend / fallend | hazard_intensity |
| Expected future change in hazard frequency | prognostizierte Veränderung der Häufigkeit | steigend / fallend | hazard_frequency |
| Current or most recent population size | Bevölkerungsgröße |  | population |
| Vulnerable population groups most exposed | gefährdete Bevölkerungsgruppen | Multi-Value-Field | exposed_population |
| City Location | Längengrade / Breitengrade | unvollständig | *fällt weg / wird gesplittet* |

In [248]:
hazard_df = pd.read_csv('2023_Cities_Climate_Hazards_20250506.csv',
                        usecols=['CDP Region',
                                 'Country/Area',
                                 'City',
                                 'Organization Name',
                                 'Climate-related hazards',
                                 'Current magnitude of impact of hazard',
                                 'Current probability of hazard',
                                 'Proportion of population exposed to hazard',
                                 'Expected future change in hazard intensity',
                                 'Expected future change in hazard frequency',
                                 'Current or most recent population size',
                                 'Vulnerable population groups most exposed',
                                 'City Location'
                                ],
                        dtype={'Current or most recent population size': 'int32'}
                       )
hazard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5015 entries, 0 to 5014
Data columns (total 13 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Organization Name                           5015 non-null   object
 1   City                                        2714 non-null   object
 2   Country/Area                                5015 non-null   object
 3   CDP Region                                  5015 non-null   object
 4   Climate-related hazards                     5015 non-null   object
 5   Vulnerable population groups most exposed   4876 non-null   object
 6   Proportion of population exposed to hazard  4675 non-null   object
 7   Current probability of hazard               4932 non-null   object
 8   Current magnitude of impact of hazard       4930 non-null   object
 9   Expected future change in hazard intensity  4735 non-null   object
 10  Expected future change i

In [249]:
head_df = hazard_df.head(8).sort_values(by=['City']).sort_index().copy()
head_df.index = [''] * len(head_df)
head_df

,Organization Name,City,Country/Area,CDP Region,Climate-related hazards,Vulnerable population groups most exposed,Proportion of population exposed to hazard,Current probability of hazard,Current magnitude of impact of hazard,Expected future change in hazard intensity,Expected future change in hazard frequency,Current or most recent population size,City Location
,Municipality of Faro,Faro,Portugal,Europe,Water stress,Children and youth; Elderly; Low-income households; Marginalized/minority communities; Outdoor workers; Vulnerable health groups,30-40%,Medium High,Medium High,Increasing,Increasing,67622,POINT (-7.93044 37.0194)
,Goicoechea,NaN,Costa Rica,Latin America,River flooding,Low-income households,10-20%,Medium,Medium,Do not know,Do not know,133557,NaN
,Delicias,NaN,Mexico,Latin America,Water stress,Children and youth; Elderly; Marginalized/minority communities; Outdoor workers,40-50%,Medium,Medium,NaN,Increasing,150506,NaN
,Ville de Maroua,NaN,Cameroon,Africa,Increased water demand,Children and youth; Elderly; Indigenous peoples; Low-income households; Marginalized/minority communities; Vulnerable health groups; Women and girls,60-70%,High,High,Increasing,NaN,700000,NaN
,Metropolitan Municipality of Rouen,ROUEN Cedex,France,Europe,Heavy precipitation,"Other, please specify: Ensemble de la population",Data is not available,Medium High,Medium High,Increasing,Increasing,494299,NaN
,"City of St. Petersburg, FL",St. Petersburg,United States of America,United States of America,Extreme wind,Elderly; Low-income households; Marginalized/minority communities; Outdoor workers; Vulnerable health groups,40-50%,Medium,Medium High,Increasing,Increasing,270000,NaN
,Hang Tuah Jaya Municipal Council,NaN,Malaysia,Southeast Asia,Infectious disease,Children and youth; Elderly,NaN,Medium,Medium Low,Increasing,NaN,188857,NaN
,Municipality of Tirana,Tirana,Albania,Europe,"Other, please specify: Air Quality","Other, please specify: All population is exposed and endangered from low air quality.",60-70%,Medium High,High,Increasing,Increasing,850530,POINT (19.8187 41.3275)


<br>

## 2. Konvertieren, auslagern, bereinigen

### 2.1. Namen ergänzen
- Wenn die Spalte "City" leer (NULL) ist, wird mit "Organization Name" aufgefüllt
- Spalte mit Organisationsnamen wird anschließend gelöscht

In [250]:
# Wenn City leer, dann verwende Organization
hazard_df['City'] = hazard_df['City'].fillna(hazard_df['Organization Name'])

# Organization-Spalte entfernen
hazard_df.drop(columns=['Organization Name'], inplace=True)

### 2.2. Koordinaten splitten
Die Koordinaten-Strings in "City Location" werden in zwei Float-Spalten aufgeteilt.
- Geo-Koordinaten liegen im WKT-Format (Well-Known Text) vor - z.B. `POINT (-7.93044 37.0194)`
- Ein Regex-Ausdruck extrahiert die Koordinaten aus dem WKT-Format (KI-generiert)
- Die Methode `.str.extract()` gibt die extrahierten Werte als Strings zurück (dtype object)
  - Durch `.astype('float32')` wird das Endergebnis nach float32 konvertiert
  - Spalten lon und lat werden erzeugt und mit den Koordinaten gefüllt
- Ursprüngliche Spalte mit kombiniertem Koordinaten-String wird gelöscht

---

| Teilregex (WKT)    | Erklärung                                                                   |
|--------------------|-----------------------------------------------------------------------------|
| `POINT`            | Sucht exakt nach der Zeichenfolge "POINT" (Groß-/Kleinschreibung beachten!) |
| `\s*`              | **0 oder mehr** Leerzeichen (Leerzeichen, Tabs, Zeilenumbrüche)             |
| `$$`               | Sucht nach der öffnenden Klammer `(` (muss mit `\` escaped werden)          |
| `\s*`              | **0 oder mehr** Leerzeichen nach der Klammer                                |
| `([-+]?\d*\.?\d+)` | **Gruppe 1**: Erfasst eine Dezimalzahl (mit optionalem Vorzeichen)          |
| `\s+`              | **1 oder mehr** Leerzeichen als Trennzeichen zwischen den Koordinaten       |
| `([-+]?\d*\.?\d+)` | **Gruppe 2**: Erfasst die zweite Dezimalzahl (wie Gruppe 1)                 |
| `\s*`              | **0 oder mehr** Leerzeichen vor der schließenden Klammer                    |
| `$$`               | Sucht nach der schließenden Klammer `)`                                     |


In [251]:
# Extrahieren der Koordinaten mit Pandas Regex und in float32 konvertieren
coords = hazard_df['City Location'].str.extract(
    r'POINT\s*\(\s*([-+]?\d*\.?\d+)\s+([-+]?\d*\.?\d+)\s*\)'
)  # Rückgabe als String
hazard_df['lon'] = coords[0].astype('float32')
hazard_df['lat'] = coords[1].astype('float32')

# "City Location" aus hazard_df entfernen
hazard_df.drop(columns='City Location', inplace=True)

### 2.3. Spalten umbenennen
DataFrame Spalten werden wegen der Übersichtlichkeit umbenannt.

In [252]:
# Spaltenumbenennung
hazard_df.rename(columns={'CDP Region': 'region',
                          'City': 'city',
                          'Country/Area': 'country',
                          'Climate-related hazards': 'hazards',
                          'Current magnitude of impact of hazard': 'hazard_impact',
                          'Current probability of hazard': 'hazard_probability',
                          'Proportion of population exposed to hazard': 'hazard_exposion',
                          'Expected future change in hazard intensity': 'hazard_intensity',
                          'Expected future change in hazard frequency': 'hazard_frequency',
                          'Current or most recent population size': 'population',
                          'Vulnerable population groups most exposed': 'exposed_population'}
                 , inplace=True)

### 2.4. Auslagerung: exposed_population
Spalte exposed_population ist ein Multi-Value-Field und wird in einen separaten DataFrame ausgelagert: `exposed_popul_df`.
- Die Spalte exposed_population nimmt Bezug auf hazards (Climate-related hazards) und ist redundant zu city
- Wichtig ist hier nur der eindeutige Bezug zur city, die hazard-Bezüge sind für spätere Betrachtungen irrelevant
- Spalte exposed_population wird gesplittet und im separaten Zuordnungs-Frame eindeutig gespeichert - mit City als Schlüsselspalte
- Spalte exposed_population wird aus dem Ursprungs-Frame gelöscht

In [253]:
# Redundante Einträge zeigen / Beispiel Empedrado
hazard_df[hazard_df.city.isin(['Empedrado'])][['city', 'exposed_population', 'hazards']]

,city,exposed_population,hazards
566,Empedrado,"Children and youth; Other, please specify: Personas con discapacidades; Personas que viven en viviendas de calidad inferior.; Women and girls","Other, please specify: Rayos / Tormenta eléctrica"
661,Empedrado,"Elderly; Other, please specify: Personas con discapacidades; Personas que viven en viviendas de calidad inferior.; Women and girls",Storm
980,Empedrado,"Other, please specify: Población y turistas que utilizan las áreas de esparcimiento","Other, please specify: Erosión hídrica por creciente del río"
1042,Empedrado,"Elderly; Other, please specify: Personas con discapacidades; Personas que viven en viviendas de calidad inferior.; Women and girls",Extreme heat
1988,Empedrado,"Children and youth; Other, please specify: Personas con discapacidades; Women and girls",Storm
2041,Empedrado,"Other, please specify: Población y turistas que utilizan las áreas de esparcimiento",Drought
2105,Empedrado,"Elderly; Other, please specify: Personas con discapacidades; Familias de las zonas rurales; Women and girls",Drought
2849,Empedrado,"Other, please specify: La población alrededor y las personas que trabajan al rededor de la actividad de gestión de residuos",Fire weather (risk of wildfires)
3231,Empedrado,"Other, please specify: La población en general y las personas que trabajan al rededor de la actividad de gestión de residuos",Storm
3520,Empedrado,"Elderly; Other, please specify: Personas con discapacidades; Personas que viven en viviendas de calidad inferior.; Women and girls",Fire weather (risk of wildfires)


In [254]:
""" Spalte mit Multi-Value-Feldern aus exposed_population in einen separaten Frame auslagern """

# Kopie von city & exposed_population erzeugen
exposed_popul_df = hazard_df[['city', 'exposed_population']].copy()

# Sonderzeichen mit Semikolon ersetzen (ist als Trennzeichen enthalten)
exposed_popul_df.exposed_population = exposed_popul_df.exposed_population.str.replace('', ';')

# Zeichenketten in Listen aufteilen
exposed_popul_df.exposed_population = exposed_popul_df.exposed_population.str.split(';')

# Listen aus Spalte in mehrere Zeilen aufteilen
exposed_popul_df = exposed_popul_df.explode('exposed_population')

# Führende und nachfolgende Leerzeichen entfernen
exposed_popul_df.exposed_population = exposed_popul_df.exposed_population.str.strip()

# Zeilen-Duplikate entfernen und internen Index neu erzeugen (ohne Index-Sicherung)
exposed_popul_df = exposed_popul_df.drop_duplicates(subset=['city', 'exposed_population']).reset_index(drop=True)

# Spalte exposed_population aus Source-Frame entfernen
hazard_df = hazard_df.drop(columns=['exposed_population'])

# Redundante Einträge nach Auslagerung und erster Bereinigung zeigen / Beispiel Empedrado
exposed_popul_df[exposed_popul_df.city.isin(['Empedrado'])][['city', 'exposed_population']]


,city,exposed_population
1566,Empedrado,Children and youth
1567,Empedrado,"Other, please specify: Personas con discapacidades"
1568,Empedrado,Personas que viven en viviendas de calidad inferior.
1569,Empedrado,Women and girls
1767,Empedrado,Elderly
2324,Empedrado,"Other, please specify: Población y turistas que utilizan las áreas de esparcimiento"
3722,Empedrado,Familias de las zonas rurales
4306,Empedrado,"Other, please specify: La población alrededor y las personas que trabajan al rededor de la actividad de gestión de residuos"
4556,Empedrado,"Other, please specify: La población en general y las personas que trabajan al rededor de la actividad de gestión de residuos"
4918,Empedrado,"Other, please specify: Toda la población"


### 2.5. Bereinigung: exposed_population
Strings in `exposed_population`, werden bereinigt und vereinheitlicht.

#### 2.5.1. Zeichen entfernen
Störende und unnötige Strings, (Leer-)Zeichen und Tabs werden entfernt

In [255]:
def cleanup_strings(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """ Bereinigt String-Spalte durch Entfernen von Strings, Sonderzeichen und Leerzeichen
        Aufruf: df = cleanup_strings(df, 'column') """

    df[column] = (
        df[column]
        .str.replace('Other, please specify: ', '', case=False)
        .str.replace('.', '', regex=False)
        .str.replace('"', '', regex=False)
        .str.replace(' ,', ',', regex=False)
        .str.replace(r'\s+', ' ', regex=True)  # Mehrere Leerzeichen durch eines ersetzen
        .str.strip()
    )
    return df

# Strings in 'exposed_population' bereinigen
exposed_popul_df = cleanup_strings(exposed_popul_df, 'exposed_population')

#### 2.5.2. Groß- und Kleinschreibung
Einträge mit gleichen Inhalten aber unterschiedlicher Groß- und Kleinschreibung werden vereinheitlicht

In [256]:
def capitalize_dupes(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """ Vereinheitlicht Strings, mit gleichen Inhalten und unterschiedlicher Groß- und Kleinschreibung
        Aufruf: df = capitalize_dupes(df, 'column') """
    
    # Hilfsspalte für Vergleich erzeugen
    df['lower_str'] = df[column].str.lower()

    # Duplikate lokalisieren (case-insensitive) und als Bool zurückgeben
    dupes = df['lower_str'].duplicated(keep=False)

    # Über Index vereinheitlichen, mit erstem Buchstaben groß
    df.loc[dupes, column] = df.loc[dupes, column].str.capitalize()

    # Hilfsspalte entfernen
    df = df.drop(columns='lower_str')

    return df

# Strings in 'exposed_population' vereinheitlichen
exposed_popul_df = capitalize_dupes(exposed_popul_df, 'exposed_population')

#### 2.5.3. Unbekannte vereinheitlichen
Leere Einträge, welche ohne Text und solche mit inhaltlichem Hinweis auf "unbekannt" erhalten den String "Unknown"

In [257]:
# exposed_population 61 NULL von 5534
print (
    pd.DataFrame({
        'count': exposed_popul_df.count(),
        'dtype': exposed_popul_df.dtypes.astype(str)
    })
    .reset_index(names=['column'])
    .to_string(index=False)
)
print('')

# Einträge für "Unknown" zusammenfassen, dabei leere Felder einschließen
mask = (
    exposed_popul_df.exposed_population.isna() |          # ist NaN --> Leere Felder
    (exposed_popul_df.exposed_population.str.len() < 3 |  # weniger als 3 Zeichen
    (exposed_popul_df.exposed_population.str.contains(    # enthaltene Strings
        'please specify|not know|n/a|none specifically|none in particular'
        , case=False
    )))
)
exposed_popul_df.loc[mask, 'exposed_population'] = "Unknown"

# exposed_population ohne NULL
print (
    pd.DataFrame({
        'count': exposed_popul_df.count(),
        'dtype': exposed_popul_df.dtypes.astype(str)
    })
    .reset_index(names=['column'])
    .to_string(index=False)
)

            column  count  dtype
              city   5534 object
exposed_population   5473 object

            column  count  dtype
              city   5534 object
exposed_population   5534 object


#### 2.5.4. Buchstaben standardisieren
Einträge mit gleichen Inhalten und unterschiedlichen Buchstaben (Akzente/Umlaute) werden vereinheitlicht
  - Modulverwendung: `unidecode`
  - Felder dürfen nicht leer sein!
  - String Konvertierung

In [258]:
# Strings vereinheitlichen, mit gleichen Inhalten und unterschiedlichen Buchstaben
exposed_popul_df.exposed_population = exposed_popul_df.exposed_population.apply(unidecode).astype('string')

# String Konvertierung "city"
exposed_popul_df.city = exposed_popul_df.city.astype('string')

# exposed_population ohne NULL
print (
    pd.DataFrame({
        'count': exposed_popul_df.count(),
        'dtype': exposed_popul_df.dtypes.astype(str)
    })
    .reset_index(names=['column'])
    .to_string(index=False)
)

            column  count  dtype
              city   5534 string
exposed_population   5534 string


#### 2.5.5. "Alle" vereinheitlichen
Einträge mit Hinweis auf "*alle Menschen*" erhalten den String "All people"
  - Modulverwendung: `re`
  - Strings werden in Regex-Muster umgewandelt
  - Muster werden | (ODER) zu einem Muster kombiniert
  - Mit dem Ergebnis wird geprüft, ob ein String einem der definierten Muster entspricht

##### 2.5.5.1. In der Liste `values` werden Strings mit inhaltlichem Hinweis auf "alle Menschen" definiert.

##### 2.5.5.2. Regex-Muster generieren
```python
regex_patterns = [f'^{re.escape(v).replace(r'\*', '.*')}$' for v in values]
```
- **`re.escape(v)`**: Sonderzeichen maskieren --> `*` wird zu `\*`
- **`replace(r'\*', '.*')`**: Ersetzen von `\*` durch `.*` (Regex-Wildcard)
- **`f'^{...}$'`**: Erstellt Regex-Muster --> Anfang `^` ... Ende `$`

###### Beispiel: 
`'all*citizens'` wird zu `^all.*citizens$`  
Das passt auf `all citizens`, `all-citizens`, `all_the_citizens` usw.

##### 2.5.5.3. Regex-Muster kombinieren
```python
combined_regex = '|'.join(regex_patterns)
```
- **`'|'.join(...)`**: Verbindet einzelne Regex-Muster mit ODER `|`.  
- Das Ergebnis ist ein Regex-Muster, das auf jeden ursprünglichen String passt.

###### Beispiel:  
```
^all$|^all.*citizens$|^all.*residents$|^all.*households$|...|^all populations can be exposed$
```


In [259]:
# Einträge für "All people" zusammenfassen
values = ['all', 'all*citizens', 'all*residents', 'all*households', 'all*popullation', 'all*inhabitants',
          'all groups will be impacted', 'all residents and businesses', 'all populations can be exposed']
regex_patterns = [f'^{re.escape(v).replace(r'\*', '.*')}$' for v in values]  # Regex-Muster
combined_regex = '|'.join(regex_patterns)
mask = (
    exposed_popul_df.exposed_population
    .str.match(combined_regex, case=False, na=False)
    & (exposed_popul_df.exposed_population.str.len() <= 30)
)
exposed_popul_df.loc[mask, 'exposed_population'] = 'All people'

#### 2.5.6. Übrige zusammenfassen
Einträge für "Different groups of people" zusammenfassen
- Übrige Einträge, mit einem vorkommen von < 100 und einer Zeichenlänge > 30, erhalten den String "Different groups of people"
- Dadurch entstehende Mehrfacheinträge werden gelöscht

In [260]:
# Hilfsspalte erstellen
exposed_popul_df['str_length'] = exposed_popul_df.exposed_population.str.len()

# Filterkriterien festlegen
result = (
    exposed_popul_df
    .groupby('exposed_population')
    .agg(count=('exposed_population', 'size'), str_length=('str_length', 'first'))
    .reset_index()
    .query("count < 100 and str_length > 30")  # vorkommen < 100 & Zeichenlänge > 30
)

# Filterkriterien in Filtermaske übernehmen
mask = exposed_popul_df.exposed_population.isin(result.exposed_population)

# Einträge Umbenennen
exposed_popul_df.loc[mask, 'exposed_population'] = 'Different groups of people'

# Hilfsspalte entfernen
exposed_popul_df = exposed_popul_df.drop(columns='str_length')

# Mehrfacheinträge entfernen und Index neu erzeugen, ohne Indexkopie
exposed_popul_df = exposed_popul_df.drop_duplicates().reset_index(drop=True)

# Bereinigte Einträge zeigen / Beispiel Empedrado
exposed_popul_df[exposed_popul_df.city.isin(['Empedrado'])][['city', 'exposed_population']]

,city,exposed_population
1539,Empedrado,Children and youth
1540,Empedrado,Personas con discapacidades
1541,Empedrado,Different groups of people
1542,Empedrado,Women and girls
1737,Empedrado,Elderly
3606,Empedrado,Familias de las zonas rurales
4709,Empedrado,Toda la poblacion


#### 2.5.7. Obsolete Kategorien
Nach dem Zusammenfassen verschiedener Felder in die Kategorien 'All people', 'Different groups of people' und 'Unknown',\
stellt sich heraus, dass diese Kategorien dann obsolet sind, wenn weitere, konkrete Kategorien für eine Stadt vorhanden sind.

Beim Beispiel Empedrado ist "Different groups of people" überflüssig, weil weitere Kategorien die betroffenen Gruppen schon konkretisieren.

##### 2.5.7.1. Erstellung des Filters
Vorgehen zum entfernen überflüssiger Kategorie-Zusammenfassungen:
- Im DataFrame exposed_popul_df wird für jede 'city' geprüft, ob in 'exposed_population' sowohl mindestens ein Eintrag aus der Menge\
  {**'All people', 'Different groups of people', 'Unknown'**} als auch mindestens ein weiterer Eintrag vorkommt, der nicht zu dieser Menge gehört.
- Für jede 'city', auf die das zutrifft, sollen anschließend alle Zeilen entfernt werden, in denen 'exposed_population' einen der genannten vier Werte enthält.

In [261]:
cat_val = {'All people', 'Different groups of people', 'Unknown'}

def filter_exposed_population(df: pd.DataFrame) -> pd.DataFrame:
    """ Filtert Städte die Gruppen-Kategorien sowie andere Kategorien enthalten
        und entfernt dann die Gruppen-Kategorien (idempotent). """
    
    # Betroffene Städte identifizieren
    city_mask = (
        # Prüfen ob Gruppen-Kategorien pro Stadt vorhanden und nicht vorhanden sind (bool)
        df.groupby('city')['exposed_population']
        .agg(lambda x: x.isin(cat_val).any() and (~x.isin(cat_val)).any())
    )
    flagged_cities = city_mask[city_mask].index.tolist()
    
    # Entferne nur Gruppen-Kategorien in diesen Städten
    remove_mask = df.city.isin(flagged_cities) & df.exposed_population.isin(cat_val)

    # Maske invers anwenden und Ergebnis zurückgeben
    return df[~remove_mask].reset_index(drop=True)
    

##### 2.5.7.2. Ergebinsprüfung
Vor dem Anwenden des Filters wird geprüft, ob das Ergebnis den Anforderungen entspricht.\
Dabei fällt auf, dass zwei Städte jeweils ausschließlich zwei der Gruppen-Kategorien zugeordnet sind:

##### Helsingborg
- Different groups of people
- Unknown

##### Reykjavík
- All people
- Different groups of people

##### --> Dies steht nicht im Widerspruch zur Plausibilität.
*( Ggf. ließe sich noch "Unknown" entfernen, wenn "All people" vorhanden ist, dieser Fall tritt hier aber nicht auf. )*

In [262]:
# Filterung zur Prüfung zwischenspeichern
filtered_df = filter_exposed_population(exposed_popul_df)

# Prüfen, ob Vorgehen zum gewünschten Ergebnis führt (Beispiel)
print('Vor dem Entfernen')
print(exposed_popul_df.count())
print('')
print('Zeilen mit Gruppen-Kategorien')
print(exposed_popul_df[exposed_popul_df.exposed_population.isin(cat_val)][['city', 'exposed_population']].count())
print('')
print('Nach dem Entfernen')
print(filtered_df.count())
print('')
print('Übrige Zeilen mit Gruppen-Kategorien')
print(filtered_df[filtered_df.exposed_population.isin(cat_val)].count())
print(
    filtered_df[filtered_df.exposed_population.isin(cat_val)]
    .sort_values(by=['city', 'exposed_population'], ascending=[True, True])
    .to_string(index=False)
)

Vor dem Entfernen
city                  5251
exposed_population    5251
dtype: int64

Zeilen mit Gruppen-Kategorien
city                  325
exposed_population    325
dtype: int64

Nach dem Entfernen
city                  4960
exposed_population    4960
dtype: int64

Übrige Zeilen mit Gruppen-Kategorien
city                  34
exposed_population    34
dtype: int64
                                   city         exposed_population
                                 Aarhus Different groups of people
                                Barueri Different groups of people
                     City of Kitakyushu Different groups of people
              Cotswold District Council Different groups of people
                                 Dalian                    Unknown
                          Florianópolis Different groups of people
   Fuzhou Municipal People's Government                    Unknown
                               Göteborg Different groups of people
                            

##### 2.5.7.3. Anwenden des Filters
Nach dem Verifizieren, wird der Filter angewendet, um die überflüssigen Gruppen-Kategorien zu entfernen.

In [263]:
# Zeilen mit erstelltem Filter löschen und internen Index zurücksetzen
# exposed_popul_df = filter_exposed_population(exposed_popul_df).reset_index(drop=True)

# prüfen/zeigen
print(exposed_popul_df.count())
print('')
print(exposed_popul_df[exposed_popul_df.exposed_population.isin(cat_val)].count())

print(exposed_popul_df)
print('')
exposed_popul_df.info()

city                  5251
exposed_population    5251
dtype: int64

city                  325
exposed_population    325
dtype: int64
                                city                 exposed_population
0                               Faro                 Children and youth
1                               Faro                            Elderly
2                               Faro              Low-income households
3                               Faro  Marginalized/minority communities
4                               Faro                    Outdoor workers
...                              ...                                ...
5246                        Canberra         Different groups of people
5247  Prefeitura de Cáceres (Brasil)                            Animais
5248                         Bristol                         All people
5249                    São Leopoldo                  Frontline workers
5250                    São Leopoldo                    Outdoor workers

[5

#### 2.5.8. CSV Export
DataFrame `exposed_popul_df` wird im aufgearbeiteten Zustand als `clim_change_exposed_popul_df.csv` gespeichert.

In [264]:
exposed_popul_df.to_csv('clim_change_exposed_popul_df.csv', index=False)

### 2.6. Bereinigung: hazards
Strings in `hazards`, werden bereinigt und vereinheitlicht.

Zuvor verwendete Funktionen werden hier wiederholt eingesetzt:
- Funktion: `cleanup_strings` (bereinigen)
- Funktion: `capitalize_dupes` (Strings vereinheitlichen)
- Modul: `unidecode` (Buchstaben vereinheitlichen)

Die Prüfung der bereinigten Inhalte wird hier nicht gezeigt, weil dies schon für exposed_population ausführlich stattfand.

In [265]:
# Strings in 'hazards' bereinigen (zuvor definierte Funktion)
hazard_df = cleanup_strings(hazard_df, 'hazards')

# Strings in 'hazards' vereinheitlichen (zuvor definierte Funktion)
hazard_df = capitalize_dupes(hazard_df, 'hazards')

# Strings vereinheitlichen, mit gleichen Inhalten und unterschiedlichen Buchstaben
hazard_df.hazards = hazard_df.hazards.apply(unidecode)


### 2.7. Vereinheitlichung: hazards

#### 2.7.1. CSV Export
Im Folgenden wird eine CSV-Datei `hazards.csv` exportiert, um sie manuell an eine LLM zu übergeben, zum Zwecke der Übersetzung und Vereinheitlichung.\
*Anmerkung: Die Einbindung eines lokalen LLM Embedding-Modells war wegen Speicherplatzbeschränkungen nicht möglich.*

In [266]:
# CSV-Export
result = (
    hazard_df[hazard_df['hazards'].str.contains('', na=False)]
    .groupby('hazards').size()
    .reset_index(name='count')
)
result['hazards_new'] = ''
result[['hazards', 'hazards_new']].to_csv('hazards.csv', index=False)

# prüfen/zeigen
# print(result['hazards'].to_string(index=False).replace('\n', ''))
# print(result['hazards'].tolist())


#### 2.7.2. LLM Prompt

**Anfrage an Perplexity:**

Bitte verarbeite die angehängte `hazards.csv` mit folgenden Schritten:  
- Erfasse alle Einträge in der Spalte hazards (multilingual).
- Führe einen semantischen Vergleich durch (z.B. "Water scarcity" = "Water stress").
- Ignoriere Unterschiede zwischen Einzahl/Mehrzahl.
- Fasse ähnliche Ausdrücke zu einer Kategorie zusammen\
(z.B. "River flooding" & "River/coastal flooding" --> "River and coastal flooding").
- Weitere Kategorien bestimmst Du inhaltlich, nach obigem Prinzip.
- Kategorienamen: Englisch, max. 30 Zeichen, nur der erste Buchstabe groß\
(z.B. "Water scarcity", "Flooding rivers coastal").
- Vollständige Tabelle mit allen Originaleinträgen und der neuen Spalte: hazards, hazards_new
- Stelle bitte sicher, dass die gepostete Tabelle kein Auszug ist und nicht nur Beispiele zeigt, sondern vollständig\
ausgegeben wird und jeder Eintrag in hazards einen zugehörigen hazards_new Eintrag erhält.

**Folge-Anfrage:**

- Die Tabelle ist noch nicht vollständig. Ich benötige keinen Python-Code sondern die Vollständige Ergebnisübersicht.
- Bitte gebe die Ergebnisübersicht mit allen 203 Einträgen der hazards Spalte und der hazards_new Spalte komplett mit\
  allen Ergebnissen aus, so dass für jeden hazard Eintrag ein hazards_new Eintrag darin steht.

---

Die vom LLM erzeugte Tabelle wurde mehrfach heruntergeladen, geprüft und mittels Anpassung des Prompts mehrmals neu erzeugt.\
Die Spaltennamen der erzeugten CSV `hazards_new.csv` mussten noch angepasst werden.\
*Anmerkung: Die Ergänzung einer zur Verfügung gestellten Datei hat nicht funktioniert, weswegen dieser Weg gewählt wurde.*


In [267]:
# Vom LLM erzeugte Tabelle als hazards_new.csv heruntergeladen und Spaltennamen modifiziert
hazards_new_df = pd.read_csv('hazards_new.csv',
                             usecols=['hazards', 'hazards_new'],
                             dtype={'hazards': 'string', 'hazards_new': 'string'}
                            )
hazards_new_df.info()
hazards_new_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   hazards      203 non-null    string
 1   hazards_new  203 non-null    string
dtypes: string(2)
memory usage: 3.3 KB


,hazards,hazards_new
0,Acute Weather Events (Number of days with high wind gusts >40 and 70 km/hr,Extreme wind
1,Affaissement de terrain,Land subsidence
2,Air Quality - Wild fire smoke,Wildfire smoke
3,Air pollution,Air pollution
4,Air pollution Da Qi Wu Ran,Air pollution
...,...,...
198,geughan gihusasangeuro inhan gibansiseolyi banbogjeog pihae ganeungseong jeungga,Infrastructure damage
199,"""hongsu, taepung deungeuro inhan suhae balsaeng jeungga""",Flood and typhoon
200,jaehaeyi daegyumohwaro inhayeo jeunggadoeneun sahoe gyeongjejeog pihaeaeg jeungga,Socioeconomic impact
201,misemeonji,Air pollution


#### 2.7.3. Merge vorbereiten

**Bereinigung und Erstellung von Hilfsspalten für den Merge**

Nach der Neuerzeugung der Schlüsselspalten durch das LLM, weicht die Formatierung von den ursprünglichen\
Werten ab. Zudem wurden z.T. Anführungszeichen eingefügt. Um einen erfolgreichen Merge zu ermöglichen,\
ist es notwendig, die Strings in den Schlüsselspalten zu vereinheitlichen. Dazu kann die zuvor definierte\
Funktion `cleanup_strings` verwendet werden, um die Werte zu bereinigen.

Da nach dieser Bereinigung immer noch kleinere Abweichungen auftreten, wird in beiden DataFrames eine\
zusätzliche Hilfsspalte als neuer Schlüssel angelegt. Für diese Hilfsspalte werden die Zeichenketten\
komplett in Kleinbuchstaben umgewandelt und sämtliche Leerzeichen, Tabulatoren sowie Zeilenumbrüche\
entfernt. Durch diese weitere Standardisierung stimmen die Werte in den Hilfsschlüsselspalten überein,\
so dass ein vollständiges Matching beim Merge erreicht wird.


In [268]:
# Strings in 'hazards' bereinigen
hazards_new_df = cleanup_strings(hazards_new_df, 'hazards')

# Hilfsspalten für Merge erzeugen (Kleinschreibung & Leerzeichen, Tabs und Zeilenumbrüche entfernen)
hazard_df['hazards_mergekey'] = hazard_df['hazards'].str.replace(r'\s+', '', regex=True).str.lower()
hazards_new_df['hazards_mergekey'] = hazards_new_df['hazards'].str.replace(r'\s+', '', regex=True).str.lower()

# prüfen/zeigen
print(hazard_df[['hazards_mergekey']].drop_duplicates().sort_values('hazards_mergekey').reset_index(drop=True))
print(hazards_new_df[['hazards_mergekey']].drop_duplicates().sort_values('hazards_mergekey').reset_index(drop=True))


                                                  hazards_mergekey
0    acuteweatherevents(numberofdayswithhighwindgusts>40and70km/hr
1                                            affaissementdeterrain
2                                                 air-bornedisease
3                                                     airpollution
4                                            airpollutiondaqiwuran
..                                                             ...
197                                             water-bornedisease
198                                       waterquality(algalbloom)
199                                                    waterstress
200                                                   wilderstorms
201                                              winterfreeze-thaw

[202 rows x 1 columns]
                                                  hazards_mergekey
0    acuteweatherevents(numberofdayswithhighwindgusts>40and70km/hr
1                                     

<br>

## 3. DataFrames zusammenführen

### 3.1. merge() & combine_first()

**Zusammenführen von DataFrames mit gleichnamigen Spalten**

##### Ausgangssituation: Merge mit identischen Spaltennamen

Wenn zwei DataFrames zusammengeführt werden und beide eine Spalte mit demselben Namen enthalten,\
benennt Pandas diese Spalten automatisch um, um Namenskonflikte zu vermeiden. Dabei wird die\
betroffene Spalte aus dem ersten (linken) DataFrame automatisch mit dem Suffix **_x** versehen\
und die entsprechende Spalte aus dem zweiten (rechten) DataFrame mit dem Suffix **_y**.

##### Beispiel:
```python
merged = dataframe_1.merge(dataframe_2, on='gemeinsamer_schlüssel', how='left')
```
##### Ergebnis:
Im zusammengeführten DataFrame existieren nun die Spalten `column_x` und `column_y`\
(sowie ggf. weitere Spalten wie `column_new`, falls vorhanden).

---

##### Entstehung neuer Spalten nach dem Merge

Nach dem Zusammenführen sind im Ergebnis-DataFrame folgende Spalten vorhanden:

- `column_x` (ursprünglich aus dem ersten DataFrame)
- `column_y` (ursprünglich aus dem zweiten DataFrame)
- Ggf. weitere Spalten wie `column_new` (falls im zweiten DataFrame vorhanden)

---

##### Verwendung von `.combine_first()`

Mit dem Befehl
```python
dataframe_1['column'] = dataframe_1['column_new'].combine_first(dataframe_2['column'])
```
wird eine neue Spalte `column` im ersten DataFrame erstellt oder eine bestehende überschrieben.\
Dabei werden fehlende Werte in `column_new` durch Werte aus der entsprechenden Spalte des zweiten\
DataFrames ersetzt. Die beiden Series werden dabei anhand ihres Index kombiniert.

##### Wichtig:
Nach einem Merge existiert keine Spalte mit dem ursprünglichen Namen (`column`) mehr, sondern nur\
die umbenannten Varianten (`column_x`, `column_y`). Durch den oben genannten Befehl wird daher eine\
neue Spalte `column` angelegt oder eine bereits vorhandene überschrieben. Die automatisch umbenannten\
und erzeugten Suffix-Spalten (`column_x`, `column_y`, `column_neu`) bleiben im DataFrame bestehen,\
solange sie nicht explizit entfernt werden. Daher empfiehlt es sich, nicht mehr benötigte Spalten\
nach dem Merge gezielt zu entfernen.

*<p style="text-align: right;">Antwort von Perplexity (modifiziert)</p>*

---

### 3.2. hazard_df + hazards_new_df

In [269]:
# hazards-Spalten ausgeben
print('Spalten, die mit "hazards" beginnen:')
for col in hazard_df.filter(regex='^hazards').columns: print(col)

Spalten, die mit "hazards" beginnen:
hazards
hazards_mergekey


In [270]:
# Merge hazard_df on hazards_new_df mit Hilfsspalten
hazard_df = hazard_df.merge(hazards_new_df, on='hazards_mergekey', how='left')

# hazards-Spalten ausgeben
for col in hazard_df.filter(regex='^hazards').columns: print(col)

hazards_x
hazards_mergekey
hazards_y
hazards_new


In [271]:
# hazard_df.hazards überschreiben mit hazards_new_df.hazards_new bei Match
hazard_df['hazards'] = hazard_df['hazards_new'].combine_first(hazards_new_df['hazards'])

# hazards-Spalten ausgeben
for col in hazard_df.filter(regex='^hazards').columns: print(col)

hazards_x
hazards_mergekey
hazards_y
hazards_new
hazards


In [272]:
# Hilfsspalten in hazard_df entfernen
hazard_df = hazard_df.drop(columns=['hazards_mergekey', 'hazards_x', 'hazards_y', 'hazards_new'])

hazard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5020 entries, 0 to 5019
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                5020 non-null   object 
 1   country             5020 non-null   object 
 2   region              5020 non-null   object 
 3   hazard_exposion     4679 non-null   object 
 4   hazard_probability  4937 non-null   object 
 5   hazard_impact       4935 non-null   object 
 6   hazard_intensity    4740 non-null   object 
 7   hazard_frequency    4501 non-null   object 
 8   population          5020 non-null   int32  
 9   lon                 2273 non-null   float32
 10  lat                 2273 non-null   float32
 11  hazards             5020 non-null   string 
dtypes: float32(2), int32(1), object(8), string(1)
memory usage: 411.9+ KB


In [273]:
# Einträge ohne Match aus hazards_new_df prüfen
not_join_new = hazards_new_df[~hazards_new_df['hazards_new'].isin(hazard_df['hazards'])]
not_join_new[['hazards', 'hazards_new']].drop_duplicates().sort_values('hazards').reset_index(drop=True)

,hazards,hazards_new


In [274]:
# Einträge ohne Match aus hazard_df prüfen
not_join_old = hazard_df[~hazard_df['hazards'].isin(hazards_new_df['hazards_new'])]
not_join_old[['hazards']].drop_duplicates().sort_values('hazards').reset_index(drop=True)

,hazards


In [275]:
# ''' Suche '''
# print(
#     hazard_df[
#         hazard_df.duplicated(subset=cols, keep=False)
#         & (hazard_df.hazard_exposion != 'Data is not available')
#         & (hazard_df.city >= 'Tr')
#         & (hazard_df.city.isin(['Arequito', 'Cipolletti', 'Mendoza', 'Municipality of Guaymallén', 'Tres Arroyos', 'Totoras', 'Vicente López']))
#     ]
#     .sort_values(['city', 'hazards'])
#     [['city', 'hazards', 'hazard_exposion', 'hazard_probability', 'hazard_impact', 'hazard_intensity', 'hazard_frequency']]
#     .drop_duplicates()
#     .head(50)
#     .to_string(index=False)
# )

In [276]:
hazard_df[hazard_df.hazards.isin(['Extreme heat', 'Extreme wind', 'Thunderstorm']) & (hazard_df.city == 'Tres Arroyos')].sort_values(['city', 'hazards'])

,city,country,region,hazard_exposion,hazard_probability,hazard_impact,hazard_intensity,hazard_frequency,population,lon,lat,hazards
60,Tres Arroyos,Argentina,Latin America,90-100%,Medium,Medium,Increasing,Increasing,57611,NaN,NaN,Extreme heat
2785,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Increasing,Increasing,57611,NaN,NaN,Extreme heat
4075,Tres Arroyos,Argentina,Latin America,90-100%,Medium,Medium,Increasing,Increasing,57611,NaN,NaN,Extreme heat
4637,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Increasing,Increasing,57611,NaN,NaN,Extreme heat
1697,Tres Arroyos,Argentina,Latin America,90-100%,Medium High,Medium High,Do not know,Do not know,57611,NaN,NaN,Extreme wind
3325,Tres Arroyos,Argentina,Latin America,90-100%,Medium High,Medium High,Do not know,Do not know,57611,NaN,NaN,Extreme wind
4929,Tres Arroyos,Argentina,Latin America,90-100%,Medium High,Medium High,Do not know,Do not know,57611,NaN,NaN,Extreme wind
218,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Do not know,Do not know,57611,NaN,NaN,Thunderstorm
270,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Do not know,Do not know,57611,NaN,NaN,Thunderstorm
1805,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Do not know,Do not know,57611,NaN,NaN,Thunderstorm


<br>

## 4. Bereinigung: hazard_df

### 4.1. Redundanz bereinigen
Nach der Kategorisierung bestehen redundante Zeilen, die entfernt werden.

In [277]:
# Redundante Zeilen entfernen
hazard_df = hazard_df.drop_duplicates().reset_index(drop=True)

hazard_df[hazard_df.hazards.isin(['Extreme heat', 'Extreme wind', 'Thunderstorm']) & (hazard_df.city == 'Tres Arroyos')].sort_values(['city', 'hazards'])

,city,country,region,hazard_exposion,hazard_probability,hazard_impact,hazard_intensity,hazard_frequency,population,lon,lat,hazards
60,Tres Arroyos,Argentina,Latin America,90-100%,Medium,Medium,Increasing,Increasing,57611,NaN,NaN,Extreme heat
2446,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Increasing,Increasing,57611,NaN,NaN,Extreme heat
1550,Tres Arroyos,Argentina,Latin America,90-100%,Medium High,Medium High,Do not know,Do not know,57611,NaN,NaN,Extreme wind
214,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Do not know,Do not know,57611,NaN,NaN,Thunderstorm


### 4.2. Wiedersprüche bereinigen
Nach dem Entfernen der Duplikate, ist zu sehen, dass es noch uneindeutige\
und wiedersprüchliche Werte in den Spalten gibt, die bereinigt werden müssen.
- `hazard_exposion`
- `hazard_probability`
- `hazard_impact`
- `hazard_intensity`
- `hazard_frequency`

---
Das Ziel ist es, möglichst den jeweils "höchsten Wert" zu behalten.

Beispiel - city: 'Tres Arroyos' / hazards: 'Extreme heat'
- hazard_probability: 'Medium' und 'Low'
- hazard_impact: 'Medium' und 'Low'

In beiden Fällen soll 'Medium' behalten werden und 'Low' wegfallen.

In [278]:
hazard_df.count()

city                  4264
country               4264
region                4264
hazard_exposion       3925
hazard_probability    4182
hazard_impact         4180
hazard_intensity      4045
hazard_frequency      3874
population            4264
lon                   2193
lat                   2193
hazards               4264
dtype: int64

#### 4.2.1. Zusammenfassen & Ränge
- Einträge ohne konkrete Informationen, werden zu "Unknown" zusammengefasst
- In zusätzlichen Hilfsspalten werden die Einträge in eine Rang-Reihenfolge gebracht

In [279]:
def assign_unknown_and_rank(
    df: pd.DataFrame,
    column_map: dict,
    unknown_patterns: str,
    rank_map: dict,
    new_rank_col: str
) -> pd.DataFrame:
    """ Ersetzt NaN und vorgegebene Strings durch "Unknown" und weist Ränge zu.
        Aufruf: df = assign_unknown_and_rank(
                     df, {'hazard_xxx': 'rank_xxx'},
                     patterns_xxx, rank_map_xxx, 'rank_xxx') """
    
    # Über beide Spalten iterieren (hazard_xxx, rank_xxx)
    for col, new_col in column_map.items():
        # Zeilen markieren, wenn NaN oder Unknown-Zuordnung vorhanden
        mask = (
            df[col].isna() |
            df[col].astype(str).str.contains(unknown_patterns, case=False, na=False)
        )
        # Markierte Zeilen auf "Unknown" setzen
        df.loc[mask, col] = "Unknown"
        # Mapping auf Rang (col --> rank_map) in new_col schreiben
        df[new_col] = df[col].map(rank_map)

    return df

# Mappings für Ränge
patterns_exposion = 'Data is not available|Question not applicable'
rank_map_exposion = {
    '<10%': 1, '10-20%': 2, '20-30%': 3, '30-40%': 4, '40-50%': 5,
    '50-60%': 6, '60-70%': 7, '70-80%': 8, '90-100%': 9, 'Unknown': 0
}
patterns_probability_impact = 'Do not know|Question not applicable|Not expected to impact the jurisdiction within the next 5 years'
rank_map_probability_impact = {
    'Low': 1, 'Medium Low': 2, 'Medium': 3, 'Medium High': 4, 'High': 5, 'Unknown': 0
}
patterns_intensity_frequency = 'Do not know|Question not applicable|Not expected to happen in the future'
rank_map_intensity_frequency = {
    'Decreasing': 1, 'Increasing': 2, 'Unknown': 0
}

# Zuordnung Rang-Spalte
column_map = {
    'hazard_exposion': 'rank_exposion',
    'hazard_probability': 'rank_probability',
    'hazard_impact': 'rank_impact',
    'hazard_intensity': 'rank_intensity',
    'hazard_frequency': 'rank_frequency'
}

##### Gruppierte Ausgabe der Spalten, vor und nach dem Zusammenfassen von "Unknown" und Zuordnen der Ränge:

In [280]:
# hazard_exposion
print(
    hazard_df.sort_values(['hazard_exposion'])
    .groupby(['hazard_exposion'], dropna=False)
    .size().reset_index(name='count')
    .to_string(index=False)
)
hazard_df = assign_unknown_and_rank(
    hazard_df,
    {'hazard_exposion': 'rank_exposion'},
    patterns_exposion,
    rank_map_exposion,
    'rank_exposion'
)
print(
    hazard_df
    .groupby(['hazard_exposion', 'rank_exposion'], dropna=False)
    .size().reset_index(name='count')
    .sort_values('rank_exposion')[['rank_exposion', 'hazard_exposion']]
    .to_string(index=False)
)

        hazard_exposion  count
                 10-20%    418
                 20-30%    410
                 30-40%    285
                 40-50%    187
                 50-60%    178
                 60-70%    134
                 70-80%    190
                90-100%    842
                   <10%    430
  Data is not available    850
Question not applicable      1
                    NaN    339
 rank_exposion hazard_exposion
             0         Unknown
             1            <10%
             2          10-20%
             3          20-30%
             4          30-40%
             5          40-50%
             6          50-60%
             7          60-70%
             8          70-80%
             9         90-100%


In [281]:
# hazard_probability
print(
    hazard_df.sort_values(['hazard_probability'])
    .groupby(['hazard_probability'], dropna=False)
    .size().reset_index(name='count')
    .to_string(index=False)
)
hazard_df = assign_unknown_and_rank(
    hazard_df,
    {'hazard_probability': 'rank_probability'},
    patterns_probability_impact,
    rank_map_probability_impact,
    'rank_probability'
)
print(
    hazard_df
    .groupby(['hazard_probability', 'rank_probability'], dropna=False)
    .size().reset_index(name='count')
    .sort_values('rank_probability')[['rank_probability', 'hazard_probability']]
    .to_string(index=False)
)

                                             hazard_probability  count
                                                    Do not know    103
                                                           High   1250
                                                            Low    296
                                                         Medium    910
                                                    Medium High    997
                                                     Medium Low    358
Not expected to impact the jurisdiction within the next 5 years     11
                                        Question not applicable    257
                                                            NaN     82
 rank_probability hazard_probability
                0            Unknown
                1                Low
                2         Medium Low
                3             Medium
                4        Medium High
                5               High


In [282]:
# hazard_impact
print(
    hazard_df.sort_values(['hazard_impact'])
    .groupby(['hazard_impact'], dropna=False)
    .size().reset_index(name='count')
    .to_string(index=False)
)
hazard_df = assign_unknown_and_rank(
    hazard_df,
    {'hazard_impact': 'rank_impact'},
    patterns_probability_impact,
    rank_map_probability_impact,
    'rank_impact'
)
print(
    hazard_df
    .groupby(['hazard_impact', 'rank_impact'], dropna=False)
    .size().reset_index(name='count')
    .sort_values('rank_impact')[['rank_impact', 'hazard_impact']]
    .to_string(index=False)
)

                                                  hazard_impact  count
                                                    Do not know    109
                                                           High    928
                                                            Low    327
                                                         Medium   1066
                                                    Medium High   1037
                                                     Medium Low    443
Not expected to impact the jurisdiction within the next 5 years     13
                                        Question not applicable    257
                                                            NaN     84
 rank_impact hazard_impact
           0       Unknown
           1           Low
           2    Medium Low
           3        Medium
           4   Medium High
           5          High


In [283]:
# hazard_intensity
print(
    hazard_df.sort_values(['hazard_intensity'])
    .groupby(['hazard_intensity'], dropna=False)
    .size().reset_index(name='count')
    .to_string(index=False)
)
hazard_df = assign_unknown_and_rank(
    hazard_df,
    {'hazard_intensity': 'rank_intensity'},
    patterns_intensity_frequency,
    rank_map_intensity_frequency,
    'rank_intensity'
)
print(
    hazard_df
    .groupby(['hazard_intensity', 'rank_intensity'], dropna=False)
    .size().reset_index(name='count')
    .sort_values('rank_intensity')[['rank_intensity', 'hazard_intensity']]
    .to_string(index=False)
)

                    hazard_intensity  count
                          Decreasing    267
                         Do not know    359
                          Increasing   3146
Not expected to happen in the future     16
             Question not applicable    257
                                 NaN    219
 rank_intensity hazard_intensity
              0          Unknown
              1       Decreasing
              2       Increasing


In [284]:
# hazard_frequency
print(
    hazard_df.sort_values(['hazard_frequency'])
    .groupby(['hazard_frequency'], dropna=False)
    .size().reset_index(name='count')
    .to_string(index=False)
)
hazard_df = assign_unknown_and_rank(
    hazard_df,
    {'hazard_frequency': 'rank_frequency'},
    patterns_intensity_frequency,
    rank_map_intensity_frequency,
    'rank_frequency'
)
print(
    hazard_df
    .groupby(['hazard_frequency', 'rank_frequency'], dropna=False)
    .size().reset_index(name='count')
    .sort_values('rank_frequency')[['rank_frequency', 'hazard_frequency']]
    .to_string(index=False)
)

                    hazard_frequency  count
                          Decreasing    322
                         Do not know    368
                          Increasing   2915
Not expected to happen in the future     12
             Question not applicable    257
                                 NaN    390
 rank_frequency hazard_frequency
              0          Unknown
              1       Decreasing
              2       Increasing


#### 4.2.2. Höchsten Rang verwenden
Es werden alle Einträge geprüft, die in Kombination aus city und hazards\
mindestens einen Rang-Eintrag haben, bei dem sich min und max unterscheiden.\
Somit werden alle Einträge mit Wiedersprüchen berücksichtigt.

##### Aggregation mit Multi-Index-Columns
**Einträge filtern, die sich im höchsten und niedrigsten Rang unterscheiden**

---
```python
hazard_df
.groupby(['city', 'hazards'], dropna=False)
.agg({'rank_exposion': ['min', 'max']})
```
`groupby()` + `agg()` erzeugt einen **Multi-Index** in den Spalten
- Spalten-Level 0: Original-Spaltenname ('rank_exposion')
- Spalten-Level 1: Aggregationsfunktion ('min', 'max')

---
```python
(agg_df.xs('max', axis=1, level=1) - agg_df.xs('min', axis=1, level=1) > 0)
.any(axis=1)
```
`xs()` **Cross-Section**: Selektiert alle Spalten des Multi-Index
- `key='max'/'min'`: Sucht nach 'max'/'min'
- `axis=0`: Sucht in Zeilen
- `axis=1`: Sucht in Spalten
- `level=0`: Erste Multi-Index Spalten-Ebene ('rank_exposion')
- `level=1`: Zweite Multi-Index Spalten-Ebene ('min', 'max')

`any()` **ODER Prüfung**: mindestens ein True-Wert
- `axis=0` Prüft pro Zeile
- `axis=1` Prüft pro Spalte

---

In [285]:
# Aggregation min/max für jeden Rang
agg_df = (
    hazard_df
    .groupby(['city', 'hazards'], dropna=False)
    .agg({
        'rank_exposion': ['min', 'max'],
        'rank_probability': ['min', 'max'],
        'rank_impact': ['min', 'max'],
        'rank_intensity': ['min', 'max'],
        'rank_frequency': ['min', 'max']
    })
)

# Filter für Einträge mit mindestens einem min/max Rang-Unterschied
mask = (
    (agg_df.xs('max', axis=1, level=1) - agg_df.xs('min', axis=1, level=1) > 0)
    .any(axis=1)
)
print(
    agg_df[mask]
    .sample(10)
    .reset_index()
    .to_string(index=False)
)

           city                    hazards rank_exposion     rank_probability     rank_impact     rank_intensity     rank_frequency    
                                                     min max              min max         min max            min max            min max
       Arequito              Wildfire risk             0   9                1   1           1   1              2   2              2   2
        Mendoza              Mass movement             1   9                1   4           1   4              2   2              0   0
      Barcelona River and coastal flooding             1   6                1   2           3   4              2   2              0   2
     Birmingham               Extreme wind             0   3                4   4           2   4              2   2              2   2
       Santiago        Loss of green space             2   9                1   3           1   3              2   2              2   2
North Vancouver River and coastal flooding      

##### Gefilterte Einträge mit maximalem Rang aktualisieren
Alle Hazard-Werte innerhalb der Stadt-Gruppen werden auf den Hazard-Wert der max-Rang-Zeile gesetzt.

---

```python
grp.loc[grp[rank_col].idxmax(), hazard_col]
```
`idxmax()` **Index des Maximums**\
Findet den Index der Zeile mit dem maximalen Wert in `rank_col`\
Selektiert damit den zugehörigen `hazard_col`-Wert
- `idxmax()` gibt den Index der Zeile mit max rank_exposion zurück
- `grp.loc[...]` selektiert den zurückgegebenen Index-Wert

---

```python
df.groupby(['city', 'hazards'])[rank_col].transform('max')
```
`transform()` **Gruppenweite Transformation**\
Berechnet den Gruppen-Maximalwert und behält die Original-Indexstruktur bei\
Erzeugt eine Spalte mit dem Gruppen-Maximum für jede Zeile

---

```python
df.loc[not_max_rank_mask, hazard_col] = # Zuweisung markierter Zeilen
```
`loc[]` **Label-basierte Selektion** - Selektiert Zeilen/Spalten durch:
- Boolsche Maske (`not_max_rank_mask`) für Zeilen
- Spaltenname (`hazard_col`) für Spalten

---

**Beispiel:**

```
| city   | hazards | rank_exposion | hazard_exposion |
|--------|---------|---------------|-----------------|
| Berlin | Fire    | 3             | High            |
| Berlin | Fire    | 1             | Medium          |
| Berlin | Fire    | 2             | Medium-High     |
```

##### Schritt 1: `idxmax()` findet max-Zeile
- Maximaler `rank_exposion` in Berlin-Fire Gruppe: 3
- Zugehöriger `hazard_exposion`: "High"

##### Schritt 2: `transform()` berechnet Gruppen-Maxima
```
| city   | hazards | rank_exposion | transform(max) |
|--------|---------|---------------|----------------|
| Berlin | Fire    | 3             | 3              |
| Berlin | Fire    | 1             | 3              |
| Berlin | Fire    | 2             | 3              |
```

##### Schritt 3: `loc[]` aktualisiert nicht-max Zeilen
```
| city   | hazards | rank_exposion | hazard_exposion |
|--------|---------|---------------|-----------------|
| Berlin | Fire    | 3             | High            |  # bleibt unverändert
| Berlin | Fire    | 1             | High            |  # aktualisiert
| Berlin | Fire    | 2             | High            |  # aktualisiert
```

*<p style="text-align: right;">Antwort von Perplexity (modifiziert)</p>*

---

In [286]:
def update_hazard_values(df: pd.DataFrame) -> pd.DataFrame:
    """ Aktualisiert die Hazard-Spalten in allen Zeilen außer denen mit maximalem Rang 
        auf den Wert aus der Zeile mit maximalem Rang pro Gruppe. """
    
    # Mapping von Rang- zu Hazard-Spalten
    rank_hazard_pairs = [
        ('rank_exposion', 'hazard_exposion'),
        ('rank_probability', 'hazard_probability'),
        ('rank_impact', 'hazard_impact'),
        ('rank_intensity', 'hazard_intensity'),
        ('rank_frequency', 'hazard_frequency')
    ]

    # Pro Rang-Hazard-Paar verarbeiten
    for rank_col, hazard_col in rank_hazard_pairs:
        # Maximalen Hazard-Wert pro Gruppe ermitteln
        max_values = (
            df.groupby(['city', 'hazards'], group_keys=False)
            [[rank_col, hazard_col]]
            .apply(lambda grp: grp.loc[grp[rank_col].idxmax(), hazard_col])
            .reset_index(name='max_hazard')
        )

        # Maske: Alle Zeilen außer denen mit maximalem Rang
        max_rank = df.groupby(['city', 'hazards'])[rank_col].transform('max')
        not_max_rank_mask = df[rank_col] != max_rank
        
        # Merge mit Originaldaten
        df = df.merge(max_values, on=['city', 'hazards'], how='left')
        
        # Aktualisieren
        df.loc[not_max_rank_mask, hazard_col] = df.loc[not_max_rank_mask, 'max_hazard']
        df = df.drop(columns='max_hazard')

    return df

# Anwenden
updated_hazard_df = update_hazard_values(hazard_df)

# Rang-Spalten löschen
updated_hazard_df = updated_hazard_df.drop(columns=['rank_exposion', 'rank_probability', 'rank_impact', 'rank_intensity', 'rank_frequency'])

# Überprüfen der Änderungen (exemplarisch)
print(
    updated_hazard_df
    .sample(10)
    .groupby(['city', 'hazards'], dropna=False)
    .agg({
        'hazard_exposion': ['first', 'last'],
        'hazard_probability': ['first', 'last'],
        'hazard_impact': ['first', 'last'],
        'hazard_intensity': ['first', 'last'],
        'hazard_frequency': ['first', 'last']
    })
    .to_string(index=False)
)

print('')
updated_hazard_df.info()

hazard_exposion         hazard_probability             hazard_impact             hazard_intensity            hazard_frequency           
          first    last              first        last         first        last            first       last            first       last
        90-100% 90-100%               High        High           Low         Low       Increasing Increasing       Increasing Increasing
        Unknown Unknown               High        High          High        High       Increasing Increasing       Increasing Increasing
        90-100% 90-100%               High        High          High        High       Increasing Increasing       Increasing Increasing
         10-20%  10-20%               High        High   Medium High Medium High       Increasing Increasing       Increasing Increasing
        Unknown Unknown                Low         Low           Low         Low       Increasing Increasing       Increasing Increasing
        Unknown Unknown               Hig

#### 4.2.3. Duplikate entfernen
Durch den vorangegangenen Prozess sind Duplikate entstanden
- Duplikate exemplarisch ausgeben
- Duplikate löschen
- Änderungen übernehmen
- Kontrollausgabe

In [287]:
# Duplikate ausgeben
print(
    updated_hazard_df[updated_hazard_df.duplicated(keep=False)]
        [['city', 'hazards', 'hazard_exposion', 'hazard_probability',
        'hazard_impact', 'hazard_intensity', 'hazard_frequency', 'lon', 'lat', 'region']]
        .head(10)
        # .sort_values(['city', 'hazards'])
        .to_string(index=False)
)

# Duplikate löschen
updated_hazard_df = updated_hazard_df.drop_duplicates()

           city        hazards hazard_exposion hazard_probability hazard_impact hazard_intensity hazard_frequency        lon      lat        region
        Mendoza   Extreme heat         90-100%               High          High       Increasing       Increasing -68.845802 -32.8894 Latin America
     Cipolletti Water scarcity         Unknown               High   Medium High       Increasing       Increasing        NaN      NaN Latin America
        Mendoza          Storm         90-100%             Medium        Medium       Increasing       Increasing -68.845802 -32.8894 Latin America
General Lavalle   Strong waves         Unknown               High          High       Increasing       Increasing        NaN      NaN Latin America
  Lujan de Cuyo   Extreme heat         Unknown        Medium High   Medium High       Increasing       Increasing        NaN      NaN Latin America
General Lavalle        Drought         Unknown         Medium Low   Medium High       Increasing       Increasin

In [288]:
hazard_df = updated_hazard_df.reset_index(drop=True)
hazard_df.info()
print('')
print(
    hazard_df[(hazard_df['city'].str.len() < 30) & (hazard_df['region'].str.len() < 30)]
    [['city', 'region', 'hazards', 'hazard_exposion', 'hazard_probability',
      'hazard_impact', 'hazard_intensity', 'hazard_frequency', 'lon', 'lat']]
    .sample(10)
    .sort_values(['city', 'hazards'])
    .to_string(index=False)
)
print('')
hazard_df[hazard_df.hazards.isin(['Extreme heat', 'Extreme wind', 'Thunderstorm']) & (hazard_df.city == 'Tres Arroyos')].sort_values(['city', 'hazards'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4010 entries, 0 to 4009
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                4010 non-null   object 
 1   country             4010 non-null   object 
 2   region              4010 non-null   object 
 3   hazard_exposion     4010 non-null   object 
 4   hazard_probability  4010 non-null   object 
 5   hazard_impact       4010 non-null   object 
 6   hazard_intensity    4010 non-null   object 
 7   hazard_frequency    4010 non-null   object 
 8   population          4010 non-null   int32  
 9   lon                 2071 non-null   float32
 10  lat                 2071 non-null   float32
 11  hazards             4010 non-null   string 
dtypes: float32(2), int32(1), object(8), string(1)
memory usage: 329.1+ KB

                  city                   region                    hazards hazard_exposion hazard_probability hazard_impact hazard_

,city,country,region,hazard_exposion,hazard_probability,hazard_impact,hazard_intensity,hazard_frequency,population,lon,lat,hazards
60,Tres Arroyos,Argentina,Latin America,90-100%,Medium,Medium,Increasing,Increasing,57611,NaN,NaN,Extreme heat
1483,Tres Arroyos,Argentina,Latin America,90-100%,Medium High,Medium High,Unknown,Unknown,57611,NaN,NaN,Extreme wind
212,Tres Arroyos,Argentina,Latin America,90-100%,Low,Low,Unknown,Unknown,57611,NaN,NaN,Thunderstorm


## 5. Koordinaten ergänzen
Die Geo-Koordinaten sind nicht vollständig und müssen ergänzt werden.\
Es gibt city-Einträge mit Koordinaten, die wiederum bei Einträgen mit der gleichen city fehlen.

### 5.1. Koordinaten ermitteln
Städte mit unterschiedlichen Koordinaten oder vorhandenen und fehlenden\
Koordinaten erhalten bei vorhandenen Koordinaten den jeweiligen Mittelwert.


In [289]:
df = hazard_df[['city', 'lat', 'lon']].copy()

# Lat und Lon zu Tupeln in neuer Spalte kombinieren
df['coords'] = list(zip(df['lat'], df['lon']))

# Anzahl unterschiedlicher Koordinaten für jede city
coords_nunique = df.groupby('city')['coords'].transform('nunique')

# Mindestens zwei verschiedene Koordinaten
mask_coords = coords_nunique != 1

# Boolsche-Maske für Zeilen mit beiden Werten not-null
has_valid_coords = df[['lat', 'lon']].notnull().all(axis=1)

# Mindestens eine vollständige Koordinate pro Stadt
mask_valid = df.groupby('city')['lat'].transform(lambda x: has_valid_coords[x.index].any())

# Beide Masken anwenden
city_coords = (
    df[mask_coords & mask_valid]
    # Gruppenweise Mittelwert-Spalten hinzufügen
    .assign(
        lat_mean=lambda d: d.groupby('city')['lat'].transform('mean'),
        lon_mean=lambda d: d.groupby('city')['lon'].transform('mean')
    )
    .drop(columns='coords')
    .drop_duplicates()
    .sort_values(['city', 'lat', 'lon'])
)

# Ausgabe
print(city_coords.to_string(index=False))

        city        lat        lon   lat_mean   lon_mean
     Abidjan   5.333690  -4.003310   5.351230  -4.006637
     Abidjan   5.360000  -4.008300   5.351230  -4.006637
       Accra   5.565430  -0.168190   5.565430  -0.168190
       Accra        NaN        NaN   5.565430  -0.168190
     Belfast  44.419998 -69.000000  44.419998 -69.000000
     Belfast        NaN        NaN  44.419998 -69.000000
       Dakar  14.752900 -17.399700  14.760633 -17.377234
       Dakar  14.764500 -17.365999  14.760633 -17.377234
    Istanbul  40.918400  29.000000  40.975548  28.986254
    Istanbul  41.008202  28.978399  40.975548  28.986254
        Lima -12.043300 -77.028297 -12.039767 -76.977966
        Lima -12.032700 -76.877296 -12.039767 -76.977966
      London  51.500000  -0.250000  51.502743  -0.152103
      London  51.504799  -0.078680  51.502743  -0.152103
      Milano  45.471699   9.188460  45.742435   9.104916
      Milano  45.802601   9.086350  45.742435   9.104916
 San Antonio  29.424101 -98.493

### 5.2. Koordinaten übernehmen
Vorhandene oder fehlende Koordinaten pro Stadt durch die Mittelwerte ersetzen.

In [290]:
# Merge der Mittelwerte
hazard_df = hazard_df.merge(
    city_coords[['city', 'lat_mean', 'lon_mean']].drop_duplicates(),
    on='city',
    how='left'
)

# Aktualisierung der Koordinaten
hazard_df['lat'] = hazard_df['lat'].combine_first(hazard_df['lat_mean'])
hazard_df['lon'] = hazard_df['lon'].combine_first(hazard_df['lon_mean'])

# Bereinigung
hazard_df = hazard_df.drop(columns=['lat_mean', 'lon_mean'])

# Ergebnis prüfen
print(
    hazard_df[['city', 'lat', 'lon']]
        .merge(
            city_coords[['city']]
                .drop_duplicates(),
                on='city',
                how='inner')
        .drop_duplicates()
        .sort_values('city')
        .to_string(index=False)
)

        city        lat        lon
     Abidjan   5.360000  -4.008300
     Abidjan   5.333690  -4.003310
       Accra   5.565430  -0.168190
     Belfast  44.419998 -69.000000
       Dakar  14.764500 -17.365999
       Dakar  14.752900 -17.399700
    Istanbul  41.008202  28.978399
    Istanbul  40.918400  29.000000
        Lima -12.032700 -76.877296
        Lima -12.043300 -77.028297
      London  51.504799  -0.078680
      London  51.500000  -0.250000
      Milano  45.802601   9.086350
      Milano  45.471699   9.188460
 San Antonio  29.424101 -98.493599
San Fernando  16.370001 120.190002
San Fernando -34.585899 -70.990799
    Santiago -33.456902 -70.648300
    Santiago -33.448898 -70.669296
    Santiago -33.452099 -70.660904


### 5.3. Fehlende Koordinaten
Es fehlen noch Koordinaten für etwa die Hälfte aller Einträge.

In [291]:
# Städte ohne Koordinaten
print(
    hazard_df[hazard_df['lat'].isna()]
        [['city', 'lat', 'lon']]
        .drop_duplicates()
        .sample(5)
        .to_string(index=False)
)
hazard_df[['city', 'lat', 'lon']].count()

                       city  lat  lon
                  Pontianak  NaN  NaN
                    Mbabane  NaN  NaN
     Municipalidad de Cañas  NaN  NaN
Municipalidad de Montecarlo  NaN  NaN
                      Bursa  NaN  NaN


city    4010
lat     2085
lon     2085
dtype: int64

<br>

## 5. CSV Export
DataFrame `hazard_df` wird im aufgearbeiteten Zustand als `clim_change_hazard_df.csv` gespeichert.

In [292]:
hazard_df.to_csv('clim_change_hazard_df.csv', index=False)